In [42]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_core.runnables import RunnablePassthrough


#### Prompt for fav_and_least_fav_product_type_promotional_text_template

In [68]:
template = """You are an Expert in writing promotional text and offer texts personalized for a given user. 
you are given some information about the customer and you have to write a promotional text for a product of the favourite product type and a product of the least favourite product type to boost their sales.
Refer to the offer_context for most common product of each type. Also you can create similar product for a given type. ALWAYS PROVIDE A PRODUCT NAME IN THE PROMOTIONAL TEXT. 
Here are the information all time favourite product categories of the customer:
{customer_info}: 
input format for customer_info= "{{Favourite product type: {{product_type}}, Least Favourite product type: {{product_type}},}}"
}}"

Influence by below sample offers for different product categories to create example promotional text based on the product types, be creative and use more words:
{offer_context}

based on above information provide the promotional text for the favourite product type and least favourite product type.

Final JSON output format= 
"{{Favourite product type: GENERATED PROMOTIONAL TEXT FOR FAVOURITE PRODUCT TYPE,
Least Favourite product type: GENERATED PROMOTIONAL TEXT FOR LEAST FAVOURITE PRODUCT TYPE,}}"

No need opening or ending paragraph, provide the JSON file ONLY.

"""

fav_and_least_fav_product_type_promotional_text_template = ChatPromptTemplate.from_template(template)


In [63]:
#### define retriever
pdf_loader = PyPDFDirectoryLoader('../promotional texts')

docs = pdf_loader.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=20)
split_docs = text_splitter.split_documents(docs)

len(split_docs)



6

In [64]:

embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embeddings,
)

# save the vector store to disk
vector_store.save_local("vector_stores/promotional_texts")



c:\Users\User\miniconda3\envs\featuren\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [65]:
# load the vector store from disk
vector_store = FAISS.load_local(folder_path="vector_stores/promotional_texts", embeddings=embeddings,allow_dangerous_deserialization=True)

retriever_promotional_texts = vector_store.as_retriever()

In [66]:
# get customer_info
import pandas as pd

def get_customer_info_fav_and_leastfav_info(customer_info_csv_path: str, customer_code: str):
    df = pd.read_csv(customer_info_csv_path)

    customer_df = df[df['customer_code'] == customer_code]

    customer_df['total_sales_quantity_breakdown'] = customer_df['total_sales_quantity_breakdown'].str.rstrip('%').astype(float)

    favorite_product_type = customer_df.loc[customer_df['total_sales_quantity_breakdown'].idxmax()]['level_1']

    least_favorite_product_type = customer_df.loc[customer_df['total_sales_quantity_breakdown'].idxmin()]['level_1']

    result = {
        'Favourite product type': favorite_product_type,
        'Least Favourite product type': least_favorite_product_type
    }
    
    return result

In [67]:
# test
result = get_customer_info_fav_and_leastfav_info("../synthesized_data/total_sales_quantity_breakdown_by_department_item_category_df.csv", 'customer_code_1')
print(result)

{'Favourite product type': 'fresh-fruits', 'Least Favourite product type': 'household-baby_needs'}


C:\Users\User\AppData\Local\Temp\ipykernel_26316\1478989567.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_df['total_sales_quantity_breakdown'] = customer_df['total_sales_quantity_breakdown'].str.rstrip('%').astype(float)


In [70]:
llm = ChatOpenAI(api_key='sk-proj-tiF6IMRa0jWHMbLr759UT3BlbkFJGxs97SwfOLb4VvvtxvVT', temperature=0.90, model_kwargs={"top_p": 0.9})

rag_chain_fav = (
    {'customer_info': RunnablePassthrough(), 'offer_context': retriever_promotional_texts}
    | fav_and_least_fav_product_type_promotional_text_template
    | llm
    | StrOutputParser()
)

RESPONSE = rag_chain.invoke(str(result))

print(RESPONSE)


{
"Favourite product type": "Celebrate Your Health! Get 20% off on a bundle of fresh fruits with our 'Fruitful Delights' offer. Stock up on your favorite fruits and enjoy the goodness of nature.",
"Least Favourite product type": "Give 'Tiny Tots Essentials' Another Try! Enjoy a special 25% discount on your next purchase of household and baby needs. We're sure you'll find something you love this time!"
}


In [72]:
def get_customer_info_highV_and_lowV_info(highV_df: str, lowV_df: str, customer_code: str, month:int):
    highV_df = pd.read_csv(highV_df)
    lowV_df = pd.read_csv(lowV_df)

    highV_filtered = highV_df[(highV_df['customer_code'] == customer_code) & (highV_df['month'] == month)]
    lowV_filtered = lowV_df[(lowV_df['customer_code'] == customer_code) & (lowV_df['month'] == month)]
    
    # Get the highest volume product type
    highest_volume_product = highV_filtered.loc[highV_filtered['volume'].idxmax()]['department_item_category']
    
    # Get the lowest volume product type
    lowest_volume_product = lowV_filtered.loc[lowV_filtered['volume'].idxmin()]['department_item_category']
    
    # Create the JSON output
    customer_info = {
        "Highest volume product type": highest_volume_product,
        "Lowest volume product type": lowest_volume_product
    }
    
    return customer_info
    


In [73]:
# test
res = get_customer_info_highV_and_lowV_info("../synthesized_data/top3_department_item_category.csv", "../synthesized_data/bottom3_department_item_category.csv", 'customer_code_1', 1)

print(res)

{'Highest volume product type': 'animal products-frozen_meat', 'Lowest volume product type': 'Hygiene-beauty_and_personal_care'}


In [79]:
template = """You are an Expert in writing promotional text and offer texts personalized for a given user. 
You are given some information about the customer's transaction history and you have to write a promotional text for a product of the highest volume sales product type and a product of the lowest volume sales product type to boost their sales.
Refer to the offer_context for the most common product of each type. You can also create similar products for a given type. ALWAYS PROVIDE A PRODUCT NAME IN THE PROMOTIONAL TEXT. 
Here is the information about the customer's transaction history: EXTREMELY IMPORTANT TO GENERATE PROMOTIONAL TEXT EMPHASIZING THE BULK BEHAVIOR OF THE CUSTOMER
{customer_info}: 
input format for customer_info= "{{Highest volume product type: {{product_type}}, Lowest volume product type: {{product_type}},}}"
}}"

Influence by the below sample offers for different product categories to create example promotional text based on the product types, be creative and use more words:
{offer_context}

Based on the above information, provide the promotional text for the highest volume product type and the lowest volume product type.

Final JSON output format= 
"{{Highest volume product type: GENERATED PROMOTIONAL TEXT FOR HIGHEST VOLUME PRODUCT TYPE,
Lowest volume product type: GENERATED PROMOTIONAL TEXT FOR LOWEST VOLUME PRODUCT TYPE,}}"

No need for an opening or ending paragraph, provide the JSON file ONLY.
"""

highest_and_lowest_volume_product_type_promotional_text_template = ChatPromptTemplate.from_template(template)


In [80]:
rag_chain_vol = (
    {'customer_info': RunnablePassthrough(), 'offer_context': retriever_promotional_texts}
    | highest_and_lowest_volume_product_type_promotional_text_template
    | llm
    | StrOutputParser()
)

RESPONSE = rag_chain_vol.invoke(str(res))

print(RESPONSE)

{
"Highest volume product type": "Enjoy the freshness of nature with our top-selling 'Frozen Meat Variety Pack'. Stock up now and save more with our exclusive bulk discount!",
"Lowest volume product type": "Give our 'Beauty and Personal Care Essentials Set' another try with a special 25% discount on your next purchase. Rediscover the benefits today!"
}
